# Breast Cancer Classification

## Load Dataset

In [14]:
from sklearn.datasets import load_breast_cancer
import numpy as np
X_raw, y_raw = load_breast_cancer(return_X_y=True)
print(X_raw.shape, y_raw.shape)

(569, 30) (569,)


### Regulate Dataset

In [17]:
X = X_raw / X_raw.max(axis=0)
y = y_raw.reshape(-1, 1)
print(X.shape, y.shape)

(569, 30) (569, 1)


## Make a Model

In [ ]:
# Define linear function
def linear(input, weight, bias):
    return input @ weight.T + bias

# Define ReLU function
def sigmoid(x):
    y = 1 / (1 + np.exp(-x))
    return y

# Redefine forward pass. Intermediate result, Z, needs to be tracked 
def forward(feature, weight, bias):
    Z = linear(feature, weight, bias)
    pred = sigmoid(Z)
    return pred
